In [1]:
# Tensorflow / Keras
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Masking,Bidirectional, LSTM, RepeatVector, Dense, TimeDistributed # for creating layers inside the Neural Network
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
# Data manipulation
import pandas as pd # for data manipulation
print('pandas: %s' % pd.__version__) # print version
import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version

# Sklearn
import sklearn
print('sklearn: %s' % sklearn.__version__) # print version
from sklearn.preprocessing import MinMaxScaler # for feature scaling

# Visualization
import plotly 
import plotly.express as px
import plotly.graph_objects as go
print('plotly: %s' % plotly.__version__) # print version

#file accessing
import os
# time stuff
from datetime import timedelta
import calendar
from matplotlib import pyplot as plt

from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_predict
from tqdm import tqdm_notebook
from pmdarima.arima import auto_arima

from itertools import product
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


Tensorflow/Keras: 2.9.0
pandas: 1.4.3
numpy: 1.23.0
sklearn: 1.1.1
plotly: 5.9.0


In [2]:
df=pd.read_csv('InSAR_data_south/displacement/export_dataframe1.csv')
df=df.set_index([df.columns[0],df.columns[1]])
df.columns=pd.to_datetime(df.columns, format='%Y%m%d')
df

2014-11-08  2014-12-02  2014-12-26  2015-02-12  \
Longitude Latitude                                                   
-120.9486 34.1347          NaN         NaN         NaN         NaN   
          34.1367          NaN         NaN         NaN         NaN   
          34.1387          NaN         NaN         NaN         NaN   
          34.1407          NaN         NaN         NaN         NaN   
          34.1427          NaN         NaN         NaN         NaN   
...                        ...         ...         ...         ...   
-117.6414 37.6236          0.0      4.8278     -1.3065         NaN   
          37.6256          0.0      4.8698     -1.2137         NaN   
          37.6276          0.0      4.8693     -1.1893         NaN   
          37.6296          0.0      4.8277     -1.1906         NaN   
          37.6316          0.0      4.8729     -1.1200         NaN   

                    2015-03-08  2015-04-01  2015-04-25  2015-05-19  \
Longitude Latitude                                                   
-120.9486 34.1347          NaN         NaN         NaN         NaN   
          34.1367          NaN         NaN         NaN         NaN   
          34.1387          NaN         NaN         NaN         NaN   
          34.1407          NaN         NaN         NaN         NaN   
          34.1427          NaN         NaN         NaN         NaN   
...                        ...         ...         ...         ...   
-117.6414 37.6236      -4.1515     -1.9999      9.9914     -3.1055   
          37.6256      -4.1588     -1.7372     10.4130     -2.9584   
          37.6276      -4.1632     -1.7107     10.4830     -2.9318   
          37.6296      -4.1240     -1.8949     10.1870     -3.0179   
          37.6316      -4.1457     -1.7033     10.5180     -2.9269   

                    2015-06-12  2015-07-06  ...  2018-10-18  2018-10-30  \
Longitude Latitude                          ...                           
-120.9486 34.1347          NaN         NaN  ...         NaN         NaN   
          34.1367          NaN         NaN  ...         NaN         NaN   
          34.1387          NaN         NaN  ...         NaN         NaN   
          34.1407          NaN         NaN  ...         NaN         NaN   
          34.1427          NaN         NaN  ...         NaN         NaN   
...                        ...         ...  ...         ...         ...   
-117.6414 37.6236     -0.32799      6.9899  ...      8.5800      11.341   
          37.6256     -0.33838      7.3402  ...      9.2515      11.996   
          37.6276     -0.34454      7.3978  ...      8.8321      11.570   
          37.6296     -0.31614      7.1245  ...     10.3320      13.073   
          37.6316     -0.33153      7.3839  ...      9.3376      12.053   

                    2018-11-11  2018-11-17  2018-11-23  2018-12-05  \
Longitude Latitude                                                   
-120.9486 34.1347          NaN         NaN         NaN         NaN   
          34.1367          NaN         NaN         NaN         NaN   
          34.1387          NaN         NaN         NaN         NaN   
          34.1407          NaN         NaN         NaN         NaN   
          34.1427          NaN         NaN         NaN         NaN   
...                        ...         ...         ...         ...   
-117.6414 37.6236       5.5704      7.6113      17.002      9.6014   
          37.6256       6.0621      8.2387      17.526      9.5942   
          37.6276       5.5897      7.8226      17.066      9.2927   
          37.6296       7.2137      9.3699      18.582     10.8610   
          37.6316       6.0863      8.3323      17.499      9.8190   

                    2018-12-17  2018-12-29  2019-01-10  2019-01-22  
Longitude Latitude                                                  
-120.9486 34.1347          NaN         NaN         NaN         NaN  
          34.1367          NaN         NaN         NaN         NaN  
          34.1387          NaN     

In [9]:
print(df.max())
print(df.min())
print(df.std())
print(df.mean())

2014-11-08     0.0000
2014-12-02     7.1085
2014-12-26     5.4012
2015-02-12    10.2810
2015-03-08     9.3765
               ...   
2018-12-05    38.2340
2018-12-17    38.6960
2018-12-29    39.1800
2019-01-10    37.3760
2019-01-22    30.4460
Length: 110, dtype: float64
2014-11-08      0.0000
2014-12-02    -19.0150
2014-12-26     -7.7813
2015-02-12    -14.4120
2015-03-08    -12.0160
                ...   
2018-12-05   -109.5700
2018-12-17   -108.9400
2018-12-29   -110.0700
2019-01-10   -110.6400
2019-01-22   -110.6300
Length: 110, dtype: float64
2014-11-08     0.000000
2014-12-02     4.915142
2014-12-26     1.505467
2015-02-12     1.832087
2015-03-08     1.809021
                ...    
2018-12-05    12.085074
2018-12-17    12.296949
2018-12-29    12.074064
2019-01-10    12.359851
2019-01-22    12.033803
Length: 110, dtype: float64
2014-11-08    0.000000
2014-12-02   -3.833238
2014-12-26   -0.780647
2015-02-12    0.236729
2015-03-08   -2.727709
                ...   
2018-12-05   -5.329

In [ ]:
tup=(-119.9588,35.185)
#tup=(-119.2990,35.8852)
#tup=(-120.2888,34.8349)
#tup=(-117.6414,37.6236)
data=df[df.index==tup].copy()
data

In [ ]:
data=data.drop(columns=data.columns[0:3],axis=1)

data=data.drop(columns=data.columns[105:108],axis=1)

In [ ]:
data=data.transpose()

data = data.set_axis(['data'], axis=1, inplace=False)
#data.insert(0,"date",value=data.index)
data=data.reset_index()
data.rename(columns={'index':'date'}, inplace=True)

In [ ]:

data.data.values[np.isnan(data.data.values)] = -2.5
data.data.values
#test=data.data.values.to_numpy().flatten() 
#test

In [ ]:
result = adfuller(data.data.values, autolag='AIC')
print(f'ADF Statistic: {result[0]}')
print(f'n_lags: {result[1]}')
print(f'p-value: {result[1]}')
for key, value in result[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')  

#It calculates the p-value and 
# compares it with a threshold value or significance level of 0.05. 
# If the p-value is less than this level, then the data is stationary; else, the differencing order is incremented by one.
##p-value <= 0.05: Reject the null hypothesis (H0), the data does not have a unit root and is stationary.

**p-value <= 0.05: Reject the null hypothesis (H0), the data does not have a unit root and is stationary.**
Data is stationary so can proceed with ARIMA or ARMA.

In [ ]:
data3=data.resample("M", on="date").agg('mean').dropna().reset_index(drop=False)
data3

Run ETS Decomposition on data (To check the seasonality in data)

In [ ]:
result=seasonal_decompose(data3.data, model='additive', period=12)

In [ ]:
result.plot()

In [ ]:
plot_pacf(result.seasonal,lags=22);
plt.show()
plot_acf(result.seasonal,lags=45);

In [ ]:

df_train=data3.iloc[:32, :].copy()
df_test=data3.iloc[32:,].copy()
print("df_train: ",df_train)
print("df_test: ",df_test)

In [ ]:
model = auto_arima(data3.data.values, start_p=1, start_q=1,
                      test='adf',
                      max_p=5, max_q=5,
                      m=1,             
                      d=1,          
                      seasonal=False,   
                      start_P=0, 
                      D=None, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

In [ ]:
model=ARIMA(df_train.data.values, order=(0,1,1))
model_fit=model.fit()
print(model_fit.summary())

In [ ]:
# Plot residual errors
residuals = pd.DataFrame(model_fit.resid)
fig, ax = plt.subplots(1,2)
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()

what are residuals??


In [ ]:
results = model.fit()

plt.figure(figsize=[15, 7.5]); # Set dimensions for figure

plt.title('-119.9588,35.185')
plt.ylabel('Displacement')
plt.xlabel('Date')
plt.xticks(rotation=90)
plt.grid(True)



plt.plot(df_train.date.values,df_train.data.values )
plt.plot(df_train.date.values, results.fittedvalues, color='red')
plt.show()

In [ ]:
MSE_error = mean_squared_error(df_train.data.values, results.fittedvalues)
print('Testing Mean Squared Error is {}'.format(MSE_error))
 #df_train.data.values

In [ ]:

plot_predict(results,1,14)

In [ ]:

##training_data = train_data['Close'].values
#test_data = test_data['Close'].values
history = [x for x in df_train.data.values]
model_predictions = []
N_test_observations = len(df_test.data.values)
for time_point in range(N_test_observations):
    model = ARIMA(history, order=(0,1,1))
    model_fit = model.fit()
    output = model_fit.forecast()
    yhat = output[0]
    model_predictions.append(yhat)
    true_test_value = df_test.data.values[time_point]
    history.append(true_test_value)
MSE_error = mean_squared_error(df_test.data.values, model_predictions)
print('Testing Mean Squared Error is {}'.format(MSE_error))

In [ ]:

plt.figure(figsize=[15, 7.5]); # Set dimensions for figure

plt.title('-119.9588,35.185')
plt.ylabel('Displacement')
plt.xlabel('Date')
plt.xticks(rotation=90)
plt.grid(True)


#plt.plot(test_set_range, model_predictions, color='blue', marker='o', linestyle='dashed',label='Predicted Price')
#plt.plot(test_set_range, test_data, color='red', label='Actual Price')
plt.plot(df_test.date.values, model_predictions, color='blue', marker='o', linestyle='dashed',label='Predicted Displacement')
plt.plot(df_test.date.values, df_test.data, color='red', label='Actual Displacement')
#plt.plot(df_train.date.values,df_train.data.values )
#plt.plot(df_train.date.values, results.fittedvalues, color='red')
plt.show()

In [ ]:
print(len( model_predictions)-len( df_test.data))

next step is to run arimax, and create a better visualization for this prediction.

precipitation and ground water